!['Relay_Problem'](Relay_Problem.png)

!['Notes Page 1'](page_1.png)

!['Notes Page 2'](page_2.png)

!['Notes Page 3'](page_3.png)

!['Notes Page 4'](page_4.png)

!['Notes Page 5'](page_5.png)

!['Notes Page 6'](page_6.png)

# Imports

In [ ]:
from __future__ import print_function
from gurobipy import *

# Define Print Functions

In [ ]:
def print_data():
    print('          ',end = '')
    for swimmer in swimmers:
        print('{:8}'.format(swimmer),end = '')
    print()
    for stroke in range(len(strokes)):
        print('{:10}'.format(strokes[stroke]),end = '')
        for swimmer in range(len(swimmers)):
            print (cost[(stroke,swimmer)],'   ',end = '')
        print()

In [ ]:
def print_data_result():
    print('          ',end = '')
    for swimmer in swimmers:
        print('{:8}'.format(swimmer),end = '')
    stroke_before = None
    for v in m.getVars():
        
            stroke = int(v.VarName[1])
            swimmer = int(v.VarName[2])
            if stroke_before != stroke:
                print()
                print('{:8}'.format(strokes[stroke]),end = '')
            stroke_before = stroke
            if v.X > 0:
                print(' [',end = '')
            else:
                print('  ',end = '')
            print(cost[(stroke,swimmer)],end = '')
            if v.X > 0:
                print('] ',end = '')
            else:
                print('  ',end = '')        
    print('\n\nFor a total:',m.objVal,'seconds')

# Input the Data

!['Relay_Problem_Data'](Relay_Problem_Data.png)

In [ ]:
strokes = ['Back','Breast','Fly','Free']
swimmers = ['Carl','Chris','David','Tony','Ken']
cost = {(0,0):37.7,(0,1):32.9,(0,2):33.8,(0,3):37.0,(0,4):35.4,
        (1,0):43.4,(1,1):33.1,(1,2):42.2,(1,3):34.7,(1,4):41.8,
        (2,0):33.3,(2,1):28.5,(2,2):38.9,(2,3):30.4,(2,4):33.6,
        (3,0):29.2,(3,1):26.4,(3,2):29.6,(3,3):28.5,(3,4):31.1}

# Review Data

In [ ]:
print_data()

# Create Optimization Model

In [ ]:
detail = False

m = Model('Relay')
m.setParam('OutputFlag', detail)

# Define the Variables 

### x(i,j) = { 1,  if stroke i is assigned to swimmer j;  0,  otherwise}
### where i is an element of strokes {0,1,2,3)
### and     j is an element of swimmers {0,1,2,3,4}

In [ ]:
x = {}
for i in range(len(strokes)):
    for j in range(len(swimmers)):
        x[i,j] = m.addVar(obj=cost[(i,j)],vtype=GRB.BINARY, name='x%d%d' % (i,j))
m.update()

In [ ]:
x

# Set up constraints

## One and Only One Swimmer Must be Assigned to Each Stroke

In [ ]:
# For each strokes (i) one at a time, the sum of the swimmers(j) must be exactly one

for i in range(len(strokes)):
   m.addConstr(quicksum([x[i,j] for j in range(len(swimmers))]) == 1, 'stroke%d' % (i))

  ## A Swimmer Can Not Swim More Than Once

In [ ]:
# For each swimmer (j) one at a time, the sum of strokes (i) must not be more than one

for j in range(len(swimmers)):
   m.addConstr(quicksum([x[i,j] for i in range(len(strokes))]) <= 1,  'swimmer%d' % (j))

# Call Optimize

In [ ]:
m.optimize()

# Look at the LP (Linear Program)

In [ ]:
 m.write('relay.lp')

# Get the Solution

In [ ]:
m.objVal

In [ ]:
m.getVars()

In [ ]:
if m.status == GRB.status.OPTIMAL:
    for v in m.getVars():
        if v.X == 1:
            stroke = int(v.VarName[1])
            swimmer = int(v.VarName[2])
            print('{:8}'.format(strokes[stroke]+':'),
                  '{:8}'.format(swimmers[swimmer]),
                  cost[(stroke,swimmer)])
    print('{:16}'.format(''),'-----\n','{:15}'.format(''),m.objVal)
else:
    print('Bummer ',m.status)


In [ ]:
print_data_result()